In [1]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots

In [2]:
ls = Vector{Any}(undef, 13) # there are 13 columns
ls[1] = String31 
for i in 2:13
    ls[i] = Float64
end

In [60]:
#load the data and orgnize 
cap_cstr = CSV.read("/home/xue.326/julia_env/LiBattery_Env/data/capacity_constraints.csv", 
                    DataFrame, 
                    header=1,
                    types=ls) |> DataFrame

distance = CSV.File("/home/xue.326/julia_env/LiBattery_Env/data/distance.csv",header=1) |> DataFrame

LCA_model = CSV.File("/home/xue.326/julia_env/LiBattery_Env/data/LCA_model314.csv",header=1) |> DataFrame

SD = CSV.File("/home/xue.326/julia_env/LiBattery_Env/data/emission_sink.csv",header=1) |> DataFrame; 

# first(cap_cstr, 5)
# first(distance, 5)
# ncol(distance)

In [8]:
first(cap_cstr, 5)

Row,Country,Li,Co,Mn,Ni,NMC111 powder,Graphite,PP,PE,Cu,Al,electricity,heat
,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,10800.0,0.0,0.0,0.0,0.0,0.0,300000.0,0.0,0.0,0.0,0.0,0.0
2,Australia,37000.0,7400.0,3.0e6,189000.0,0.0,500.0,500000.0,600000.0,920000.0,1.6e6,0.0,0.0
3,Bahrain,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2e6,0.0,0.0
4,Bolivia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Brazil,1800.0,0.0,1.2e6,135000.0,0.0,87000.0,1.9e6,3.6e6,400000.0,590000.0,0.0,0.0


In [27]:
# first(LCA_model, 5)

In [61]:
LCA_model

Row,process,input (material/kg battery),input (material/kWh battery),EF (kg CO2/material),input (material/kg NMC111 powder)
,String15,Float64,Float64,Float64,Float64
1,Li,0.00905,0.0635,0.0155,0.072
2,Co,0.051,0.358,0.492,0.203
3,Mn,0.0479,0.336,5.6,0.19
4,Ni,0.051,0.358,1.07,0.203
5,NMC111 powder,0.252,1.77,14.7,0.0
6,Graphite,0.141,0.986,4.86,0.0
7,PP,0.011,0.0774,2.53,0.0
8,PE,0.00364,0.0255,2.93,0.0
9,Cu,0.117,0.824,3.08,0.0


In [11]:
first(SD, 5)

Row,country,emission,sink ton/yr
,String31,Float64,Float64
1,Argentina,1.98e8,1.79e8
2,Australia,3.89e8,2.87e8
3,Bahrain,3.33e7,72500.0
4,Bolivia,8.97e7,1.73e8
5,Brazil,8.19e8,1.81e9


In [49]:
w_cell = 164.98                             # kg/pack
cap_cell = 23.5                             # kWh/pack
cell_demand = 6000000                       # annual demand of Li-ion battery for tesla (2M EV/yr, 3 NMC111 pack/EV)
gobal_sink = 1.53e9                         # global CO2 sequestration
global_emission = 4.75e10                   # global CO2 emission
EF_aircraft = 0.433                         # kg CO2/km*ton emission factor for freight transporation 
EF_input = LCA_model[!,"EF (kg CO2/material)"]
process = LCA_model[!,"process"]
countries = cap_cstr[!,"Country"]
n = size(countries,1)                       # No. of countries
m = size(process,1);                        # No. of processes 

In [29]:
# seperate model
ups_powder = [1, 2, 3, 4]      # upstream processes for NMC111 powder
ups_cell = [i for i in 5:10];  # upstream processes for Li-ion battery manufacturing

scaler_powder = LCA_model[1:4,"input (material/kg NMC111 powder)"]     # mineral inputs for 1 kg NMC111 powder
scaler_cell = LCA_model[!, "input (material/kg battery)"];             # inputs for 1 kg NMC111 cell

In [39]:
FU_mass = cell_demand * w_cell      # input amount per FU
input_FU = DataFrame(process = LCA_model[!,"process"], inputs = FU_mass .* LCA_model[!,"input (material/kg battery)"]);
input_FU = input_FU[!, "inputs"]
# first(input_FU, 5)

12-element Vector{Float64}:
 8.958414e6
 5.048387999999999e7
 4.741525199999999e7
 5.048387999999999e7
 2.4944975999999997e8
 1.3957307999999997e8
 1.0888679999999998e7
 3.6031631999999997e6
 1.1581596e8
 2.3658131999999997e8
 9.997788e7
 1.7124923999999996e7

--------------------

In [40]:
model = Model(Gurobi.Optimizer);
# set_optimizer_attribute(model, "NonConvex", 2);

Academic license - for non-commercial use only - expires 2023-11-24


In [41]:
#variables 
@variable(model, x[1:n, 1:m] >= 0)         # x[i,k] production amount of product k at location i
@variable(model, y[1:n, 1:n, 1:m] >= 0);   # y[i,j,k] ship product k from i to j 


# node output flow constraint
for k in 1:m
    for i in 1:n
        @constraint(model, sum(y[i,j,k] for j in 1:n) <= x[i,k])
    end
end


# NMC111 powder input flow constraint
for k in ups_powder
    for j in 1:n
        @constraint(model, sum(y[i,j,k] for i in 1:n) >= x[j,5] * scaler_powder[k])
    end
end


# Li-ion battery input flow constraint
for k in ups_cell
    for j in 1:n
        @constraint(model, sum(y[i,j,k] for i in 1:n) >= x[j,11] * scaler_cell[k])
    end
end


# production capacity constraints
for i in 1:n
    for k in 1:m
        @constraint(model, x[i,k] <= cap_cstr[!, 2:end][i,k])
    end
end


# final demand constraints
for k in 1:m
    @constraint(model, sum(x[i,k] for i in 1:n) >= input_FU[k])
end

In [54]:
proc_emi = []

for k in 1:m
    emi = sum(x[i, k] for i in 1:n)
    push!(proc_emi, emi)
end

tot_pemi = EF_input' * proc_emi;


In [63]:
trans_emi = 0
for k in 1:m
    for i in 1:n
        for j in 1:n
            trans_emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_aircraft
        end
    end
end

cell_output = sum(x[i,11] for i in 1:n)
cell_temi = trans_emi + sum(cell_output * distance[!, 2:end][i,38] for i in 1:n)

5501.18097635 y[1,2,1] + 5977.18814278 y[1,3,1] + 1065.308829624 y[1,4,1] + 1266.200209298 y[1,5,1] + 4875.16967621 y[1,6,1] + 331.80750159670004 y[1,7,1] + 8183.26438468 y[1,8,1] + 2123.891741214 y[1,9,1] + 4038.15283431 y[1,10,1] + 2955.981751935 y[1,11,1] + 5920.56513547 y[1,12,1] + 4964.301236689999 y[1,13,1] + 3753.8510283289997 y[1,14,1] + 5317.20569341 y[1,15,1] + 3575.387023947 y[1,16,1] + 6896.24991994 y[1,17,1] + 6775.75119429 y[1,18,1] + 6236.8697259400005 y[1,19,1] + 7824.325189640001 y[1,20,1] + 6903.11050184 y[1,21,1] + 4498.13683141 y[1,22,1] + 6902.65038305 y[1,23,1] + 3465.997920022 y[1,24,1] + 5048.07437021 y[1,25,1] + 5534.71773565 y[1,26,1] + 6185.037266089999 y[1,27,1] + 1491.0885342729998 y[1,28,1] + 7415.58916734 y[1,29,1] + 4472.7261441400005 y[1,30,1] + 7506.11586248 y[1,31,1] + 5718.31711397 y[1,32,1] + 3327.76462112 y[1,33,1] + 8213.70961491 y[1,34,1] + 5728.781563529999 y[1,35,1] + 5802.914324429999 y[1,36,1] + 6035.361662470001 y[1,37,1] + 3904.661346986 y[

In [62]:
distance[!, 1:end]

Row,Column1,Argentina,Australia,Bahrain,Bolivia,Brazil,Canada,Chile,China,Colombia,Congo,Cuba,Finland,France,Gabon,Germany,Ghana,India,Indonesia,Iran,Japan,Kazakhstan,Madagascar,Malaysia,Mexico,New Caledonia,Norway,Papua New Guinea,Peru,Philippines,Portugal,Russia,Saudi arabia,South Africa,South korea,Turkey,Ukraine,United Arab Emirates,United States,Zambia,Zimbabwe
,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,0.0,12704.8,13804.1,2460.3,2924.25,11259.1,766.299,18899.0,4905.06,9325.99,6826.75,13673.4,11464.9,8669.4,12279.9,8257.24,15926.7,15648.4,14403.9,18070.0,15942.5,10388.3,15941.5,8004.61,11658.4,12782.3,14284.2,3443.62,17126.1,10329.6,17335.1,13206.3,7685.37,18969.3,13230.4,13401.6,13938.5,9017.69,9230.47,8940.93
2,Australia,12704.8,0.0,10592.5,15025.3,15578.6,14151.6,12734.1,7474.36,16238.3,12002.8,16775.3,13405.5,15159.1,13163.7,14466.2,14859.6,7813.47,3457.44,10630.0,6852.33,10515.3,8831.35,4746.11,14353.7,3285.43,14342.1,2370.08,15075.0,4437.48,16150.9,9983.62,10993.0,10389.5,6832.37,12441.4,12983.4,10164.3,15184.5,10932.2,10496.6
3,Bahrain,13804.1,10592.5,0.0,13168.0,11923.8,10617.8,14465.8,5164.49,13173.4,4563.18,12334.4,4393.86,4790.88,5135.17,4377.2,5798.87,2957.08,7409.17,769.656,8141.68,2827.16,5001.18,5979.58,13777.5,13433.8,4969.62,10653.7,13996.8,7524.75,5593.87,5637.64,603.028,6948.25,7254.57,2023.76,2998.47,443.509,12156.4,5003.61,5520.25
4,Bolivia,2460.3,15025.3,13168.0,0.0,1271.64,8984.71,2295.17,17525.4,2601.49,9385.91,4480.13,11566.5,9563.62,8403.44,10345.4,7391.25,16044.5,18096.4,13504.3,16924.2,14313.6,11469.1,17944.2,6133.65,13239.0,10635.1,16077.8,1468.55,19324.7,8440.36,14909.1,12591.1,8772.15,17561.2,11888.6,11700.0,13471.6,6823.15,9750.86,9701.9
5,Brazil,2924.25,15578.6,11923.8,1271.64,0.0,9304.54,3084.2,16632.9,3234.22,8140.02,4875.15,10771.2,8625.31,7137.68,9431.66,6132.5,14775.3,17730.6,12299.7,17370.5,13311.2,10397.7,16945.6,6928.3,14351.5,9867.1,17135.7,2574.26,19317.6,7486.78,14451.5,11340.2,7777.15,17605.0,10757.2,10686.6,12214.5,7316.93,8571.87,8579.94
6,Canada,11259.1,14151.6,10617.8,8984.71,9304.54,0.0,10725.8,9385.87,6395.73,12640.0,4507.97,6245.51,6840.81,11768.4,6750.73,10205.6,11469.1,12886.3,9987.74,8082.72,8417.27,15297.9,12827.5,3626.63,11805.1,5850.39,11802.6,7815.51,11145.1,6917.86,6644.49,10718.1,15184.0,8578.95,8837.84,7744.83,10973.6,2261.54,13848.5,14485.1
7,Chile,766.299,12734.1,14465.8,2295.17,3084.2,10725.8,0.0,19630.1,4484.38,10043.2,6393.87,13818.1,11709.2,9337.21,12514.0,8804.76,16691.8,15921.0,15021.8,17342.6,16364.4,11149.9,16453.7,7357.69,11312.7,12896.0,14053.2,2965.98,17150.4,10570.9,17132.4,13863.7,8451.18,18276.1,13711.3,13753.5,14629.9,8466.86,9985.79,9705.13
8,China,18899.0,7474.36,5164.49,17525.4,16632.9,9385.87,19630.1,0.0,15516.5,9592.2,13631.0,5961.15,8022.01,10292.9,7223.9,10836.3,2983.8,4198.13,4612.32,3046.76,3325.16,8556.58,3526.81,12826.4,9028.17,6891.11,6260.74,17048.3,3102.75,9157.71,2854.62,5766.63,11237.1,2118.37,5949.88,5947.1,5000.89,11647.3,9668.54,9963.54
9,Colombia,4905.06,16238.3,13173.4,2601.49,3234.22,6395.73,4484.38,15516.5,0.0,10713.1,1921.95,10080.9,8604.83,9561.05,9245.78,8098.79,15972.0,19010.2,13161.7,14357.3,13056.7,13447.6,18954.5,3696.57,13308.4,9169.25,15777.2,1532.2,17385.1,7640.9,12665.6,12732.8,10957.9,14961.2,11349.8,10754.3,13591.0,4222.32,11436.2,11585.9


In [43]:
cap_cstr[!, 2:end]

Row,Li,Co,Mn,Ni,NMC111 powder,Graphite,PP,PE,Cu,Al,electricity,heat
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,10800.0,0.0,0.0,0.0,0.0,0.0,300000.0,0.0,0.0,0.0,0.0,0.0
2,37000.0,7400.0,3.0e6,189000.0,0.0,500.0,500000.0,600000.0,920000.0,1.6e6,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2e6,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1800.0,0.0,1.2e6,135000.0,0.0,87000.0,1.9e6,3.6e6,400000.0,590000.0,0.0,0.0
6,1540.0,4440.0,0.0,216000.0,0.0,15000.0,1.4e6,2.5e6,480000.0,2.9e6,0.0,0.0
7,18500.0,0.0,0.0,0.0,0.0,0.0,0.0,300000.0,5.7e6,0.0,0.0,0.0
8,9240.0,2000.0,1.4e6,0.0,1.0e11,850000.0,2.38e7,2.7e7,1.6e6,3.7e7,1.0e15,1.0e15
9,0.0,0.0,0.0,80000.0,0.0,0.0,200000.0,600000.0,0.0,0.0,0.0,0.0


In [44]:
cap_cstr[!, 2:13][1,2]

0.0